In [1]:
from openai import OpenAI

client = OpenAI() # requires OPENAI_API_KEY to be set

seed = 1 # global seed for repeatability

# Create tasks independently

## 1. Generate a list of popular Shopify storefronts.

In [6]:
# def generate_storefronts():
from os import listdir
from os.path import join

def storefronts_generator():
    storefront_dir = 'storefronts'
    html_files = [f for f in listdir(storefront_dir)]
    for html_file in html_files:
        with open(join(storefront_dir, html_file)) as f:
            html = f.read()
            yield {
                'store_name': html_file.split('.')[0],
                 'html': html
            }

storefronts = storefronts_generator()
print(next(storefronts)['store_name'])

Beefcake


## 2. For each storefront, retrieve the contents of the store website and get metadata

In [20]:
from bs4 import BeautifulSoup

def summarize_site(store_name, html):
    site_text = BeautifulSoup(html, "html.parser").get_text()

    messages = [
        {
            "role": "system",
            "content": """You are an award winning Shopify product manager. You have received awards and accolades for your creative, insightful, yet pragmatic thinking.

    You are tasked with reviewing website content of a Shopify storefront and better understanding their brand and product offering.
    """
        },
        {
            "role": "user",
            "content": f"""The following is the content of the {store_name} storefront. Consider the contents: What are they selling? What values do they hold as important? What is their overal tone (e.g. fun, formal, adventurous)? Do you see any themes (travel, productivity, exercise)? Is there timely, praiseworthy news?
    ```
    {site_text}
    ```

    Answer each of these questions one at a time:
    1. What are they selling?
    2. What values do they hold as important?
    3. What is their overal tone (e.g. playful, formal, adventurous, relaxing)?
    4. Do you see any themes (travel, productivity, exercise)?
    5. What do you find praiseworthy and timely? Is there any news that {store_name} would be proud of?
    6. What else do you see that is noteworthy?
    """
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    summary = response.choices[0].message.content
    
    return summary

print(summarize_site(store_name="Wickles", html="<body>We sell pickles</body>"))

1. What are they selling?
They are selling pickles.

2. What values do they hold as important?
Based on the limited content provided, it's difficult to ascertain specific values. However, given that they are selling pickles, they may value quality, flavor, and possibly tradition or nostalgia associated with pickling.

3. What is their overall tone?
The overall tone seems straightforward and to the point. It doesn't convey a specific tone like playful, formal, adventurous, or relaxing.

4. Do you see any themes?
Given the limited information, there doesn't seem to be a clear theme like travel, productivity, or exercise. However, the theme of food and culinary enjoyment may be relevant.

5. What do you find praiseworthy and timely?
It's praiseworthy that Wickles is focused on a specific product category, pickles, which can cater to a niche market of pickle enthusiasts. If there are any unique or innovative aspects to their pickles, such as special flavors or sustainable sourcing, highlig

## 3. Come up with one idea for a software plugin that could be built to increase their sales.

In [15]:
def brainstorm_plugin(store_name, summary):
    # brainstorm first and decide which item is best
    messages = []
    messages.extend([
        {
            "role": "system",
            "content": """You are an award winning Shopify product manager. You have received awards and accolades for your creative, insightful, yet pragmatic thinking. You are familiar with Shopify's plugin platform and its capabilities.

    You are tasked with reviewing website a Shopify storefront and then creating a concept for a new Shopify plugin that will help the storefront owner generate more attention, more business, and more sales. The plugin must be relatively simple to implement.
    """
        },
    {
        "role": "user",
        "content": f"""Here is a summary of the {store_name} store front.

    ```
    {summary}
    ```

    Now toss out a few ideas for plugins. Then identify the best one in terms of impact to customer, match with their brand, and ease of implementation."""
    }])
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })

    # then summarize that item
    messages.extend([
    {
        "role": "user",
        "content": f"Now take the best idea and create an in-depth analysis of just that idea. Explain the idea in detail. How would it look? How would {store_name} users interact with it? How would is make money for the store? At a high level, how would it be implemented?"
    }])
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    idea = response.choices[0].message.content
    return idea

print(brainstorm_plugin(store_name="Wickles", summary="they sell pickles"))

**Idea: Product Recommendation Engine**

**Concept:**
The Product Recommendation Engine is a powerful tool that analyzes customer behavior and purchase history to provide personalized product recommendations to visitors on the Wickles storefront. When a customer visits the website, they will see a section showcasing recommended pickle products tailored to their preferences and buying patterns.

**How it works:**
1. **Customer Data Analysis:** The plugin will analyze customer data such as browsing history, past purchases, and demographics to understand each customer's preferences.
2. **Recommendation Generation:** Based on the analysis, the plugin will generate personalized product recommendations for each visitor, showcasing a selection of pickle products that they are likely to be interested in.
3. **Display on Storefront:** The recommended products will be displayed prominently on the homepage or product pages, encouraging customers to explore and make a purchase.

**User Interaction

## 4. Generate a marketing email to see if they would be interested in letting us build something for them.

In [11]:
def generate_email(store_name, summary, idea):
    # Start by coming up with a pitch strategy
    messages = []
    messages.extend([{
        "role": "system",
        "content": """You are an highly successful marketer and content creator working for JivePlugins Inc.

    You will be given 1) a summary of a Shopify storefront and 2) a concept for a new shopify plugin. You are to work with your team to help generate marketing content that will be sent to the storefront owner promoting the plugin and encouraging them to reach out to JivePlugins for further discussion.
    """
    },
    {
        "role": "user",
        "content": f"""Here is the summary of the {store_name} storefront
    ```
    {summary}
    ```

    and here is a product idea to promote

    ```
    {idea}
    ```

    Let's first work on our approach (do not write an email). What is our strategy for enticing the storefront owner? Remember we're talking to someone that is at the executive level, so they don't want the fine-grained details. Rather, we must sell them on an exciting idea that will engage customers and boost the bottom line and then we must have a call-to action to talk further with JivePlugins about plans in implementing this."
    """
    }])
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    pitch_strategy = response.choices[0].message.content
    messages.append({
        "role": "assistant",
        "content": pitch_strategy
    })

    # next, come up with the subject line (note that the prompt still has the summary, the plugin idea, and the pitch strategy in the context window)
    messages.extend([
    {
        "role": "user",
        "content": f"What is the subject line of the email? It should be in a style that matches the vibe of the website. Don't add any commentary - just write the subject line. Don't prefix your answer with \"subject:\" - just write the subject line."
    }])
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    email_subject = response.choices[0].message.content
    messages.append({
        "role": "assistant",
        "content": email_subject
    })

    messages.extend([
    {
        "role": "user",
        "content": f"""What is the body of the email? Guidelines:
    - It should be in a style that matches the vibe of the website.
    - If possible, give a genuine compliment that links back to something we found on the website.
    - It should make use of our earlier discussion about strategy - selling them on the highlights and practicality of the plugin concept.
    - It should include a description of the plugin idea:
    - Help them understand what it will look like.
    - Help them understand how the users will engage.
    - Help them understand the bottom line for business.
    - It should have a call to action to reach out to JivePlugins for further conversation.
    - Just answer with the body:
    - Don't include any commentary before or after
    - Don't prefix your answer with the subject line again - just write the body.
    - Address the "{store_name} Team" in the salutation
    - The signiture should be from "Albert Berryman" the "Director of Innovation" of JivePlugins Inc."""
    }])
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        n=1,
        temperature=0.7,
        seed = seed
    )
    email_body = response.choices[0].message.content

    return {
        "subject": email_subject,
        "body": email_body,
    }

print(generate_email(store_name="Wickles", summary="they sell pickles", idea="picklerater"))

{'subject': '"🥒 Spice Up Your Pickle Game with Picklerater! 🌶️"', 'body': 'Hey Wickles Team,\n\nI hope this message finds you in a pickle-perfect day! 🥒\n\nI recently came across your delightful storefront and must say, your passion for pickles truly shines through! Your dedication to quality and flavor is truly inspiring.\n\nI wanted to introduce you to an exciting idea - the "picklerater" plugin. Imagine a feature on your website where customers can rate and review their favorite pickles, sharing their love for Wickles with the world. This interactive platform not only engages users but also provides you with valuable feedback to enhance your product offerings. By leveraging the "picklerater" plugin, you can deepen customer loyalty, attract new pickle enthusiasts, and ultimately boost your bottom line.\n\nI believe this innovative plugin has the potential to set Wickles apart in the market and create a buzz among your customers. I invite you to reach out to JivePlugins for a further 

## 5. Send the emails

In [13]:
def send_email(email):
    print(f"SUBJECT: {email['subject']}\n\n{email['body']}\n\n===========================================")

send_email({
    'subject': 'Picklerater',
    'body': "You should let us sell you a Pickelrater. Trust me."
})



EMAIL SENT:
SUBJECT: Picklerater

You should let us sell you a Pickelrater. Trust me.



# Assemble Workflow and Run

In [21]:
def shopify_plugin_promoter():
    storefronts = storefronts_generator()
    for storefront in storefronts:
        summary = summarize_site(store_name=storefront['store_name'], html=storefront['html'])
        idea = brainstorm_plugin(store_name=storefront['store_name'], summary=summary)
        email = generate_email(store_name=storefront['store_name'], summary=summary, idea=idea)
        send_email(email)

shopify_plugin_promoter()

EMAIL SENT:
SUBJECT: Dive into a New Virtual Swimsuit Experience with AR Try-On!

Dear Beefcake Team,

I hope this email finds you well! I recently had the pleasure of exploring your sustainable and inclusive swimsuit collection, and I must say, the commitment to body positivity and sustainability truly shines through your brand.

I am excited to present to you a cutting-edge concept that I believe aligns perfectly with Beefcake's values and vision. Introducing our Virtual Try-On Tool Analysis plugin, a revolutionary addition to your Shopify storefront that will elevate the shopping experience for your customers.

Imagine your users being able to virtually try on different swimsuit styles using augmented reality technology. With just a click of a button, they can see themselves in real-time wearing your one-piece swimsuits, gaining a realistic representation of how the swimsuit would look on them. This immersive experience not only boosts engagement on your website but also reduces ret